In [3]:
import torch
import torch.nn.functional as F

In [15]:
class ResidualNet(torch.nn.Module):
    # 需要保证输出层的大小不变，卷积核是3 ， 需要填充 padding 1  因为 7- 3 + 1 = 5
    def __init__(self,channel):
        super(ResidualNet,self).__init__()
        self.conv_1 = torch.nn.Conv2d(channel,channel,kernel_size=3,padding=1)
        self.conv_2 = torch.nn.Conv2d(channel,channel,kernel_size=3,padding=1)
        
    def forward(self,x):
        y = F.relu(self.conv_1(x))
        y = self.conv_2(y)
        return y + x

    
r = ResidualNet(2)
a = torch.Tensor([1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9]).view(1,2,3,3)
y = r(a)
y

tensor([[[[1.0973, 2.0973, 3.0973],
          [3.9774, 4.6937, 5.6348],
          [6.4430, 7.4558, 8.8629]],

         [[1.2160, 2.2160, 3.2160],
          [4.9479, 5.9718, 6.5344],
          [6.9498, 7.5582, 8.5432]]]], grad_fn=<AddBackward0>)